In [1]:
import networkx as nx
import os
import json
import re
from fractions import Fraction

In [2]:
# USER INPUTS :

# 1. Reachability graph generated from PROM exported as Petrify SG file. (WITHOUT the extension)
petrify_sgfile='small_process_inductive'

#2. ENDING EVENT(S) :
end_event=['f']

file1=petrify_sgfile+'.sg'
file3=petrify_sgfile+'_lts_sim.txt'
file4=petrify_sgfile+'_lts_Simulator.mcrl2'
file6=petrify_sgfile+'_adtmc_sim.mcrl2'
file7=petrify_sgfile+'_sdtmc_sim.prism'
file9=petrify_sgfile+'_dfs_sim.txt'

# PRE-PROCESSING THE SG FILE.

file = open(file1, 'r')
Lines = file.readlines()
#print(Lines)
file.close()
lines=[]
for x in Lines:
    y=re.sub(r'_.00', '', x)
    y=re.sub(r'_.01', '', y)
    y=re.sub(r'_.11', '', y)
    y=re.sub(r'_.04', '', y)
    y=re.sub(r'_.05', '', y)
    y=re.sub(r'taustart', 'tau', y)
    y=re.sub(r'taujoin', 'tau', y)
    y=re.sub(r'tausplit', 'tau', y)
    y=re.sub(r'taufromtree', 'tau', y)
    lines.append(y)
#print(len(lines))
#print(lines)
# for l in lines:
#     print(l)

In [3]:
# INFERING THE STATES, ACTIONS & THE INITIAL STATE OF THE MODEL.

start=lines.index('.state graph\n')+1
for i in range(len(lines)):
    if '.marking' in lines[i]:
        end=i
initial=[]
x=list(lines[end])
strin=''
for i in range(x.index('{')+1,x.index('}')):
    if x[i]!=' ':
        strin+=x[i]
initial.append(strin)
states=[]
actions=[]
for i in range(start,end):
    x=lines[i].split()
    if x[0] not in states:
        states.append(x[0])
    if x[-1] not in states:
        states.append(x[-1])
    if x[1] not in actions and x[1].lower()!='tau':
        actions.append(x[1])  
actions.append('NEWTRACESTARTING')
#print(len(states))
#print(states)
#print(actions)
NTS=[]
s1=[]
new_lines=[]
NTS=[]
ctr=1
sctr=0
for i in range(len(lines)):
    if len(lines[i].split())>2:
        if lines[i].split()[1] in end_event and lines[i].split()[0]==lines[i].split()[-1]:
            temp_l1=''
            temp_l2=''
            temp_l1=lines[i].split()[0]+' '+lines[i].split()[1]+' s'+str(len(states)+sctr)+'\n'
            temp_l2='s'+str(len(states)+sctr)+' '+lines[i].split()[1]+' '+lines[i].split()[0]+'\n'
            new_lines.append(temp_l1)
            new_lines.append(temp_l2)
            NTS.append('s'+str(len(states)+sctr))
            states.append('s'+str(len(states)+sctr))
            sctr+=1
        elif lines[i].split()[1] in end_event and lines[i].split()[0]!=lines[i].split()[-1]:
            NTS.append(lines[i].split()[-1])
            new_lines.append(lines[i])
        else:
            new_lines.append(lines[i])
    else:
        new_lines.append(lines[i])
for i in range(len(new_lines)):
    if len(new_lines[i].split())>2:
        if new_lines[i].split()[1]=='tau':
            s=[]
            s.append(new_lines[i].split()[0])
            s.append(new_lines[i].split()[1]+str(ctr))
            ctr+=1
            s.append(new_lines[i].split()[-1])
            s1.append(s)
#print(s1)
#print(new_lines,NTS)

In [4]:
# CONSTRUCTING THE ADJACENCY MATRIX.

adjacency=[]
for x in states:
    if x not in adjacency:
        y=[]
        y.append(x)
        ad=[]
        for i in range(start,end):
            l=new_lines[i].split()
            if x==l[0]:
                m=l[1]+'.'+l[2]
                ad.append(m)
        y.append(ad)
    adjacency.append(y)
#print(adjacency)                
adj={}
for i in range(len(adjacency)):
    if adjacency[i][0] not in adj:
        adj[adjacency[i][0]]={}
        for j in range(len(adjacency[i][-1])):
            if adjacency[i][-1][j].split('.')[0] not in adj[adjacency[i][0]]:
                adj[adjacency[i][0]][adjacency[i][-1][j].split('.')[0]]=[]
                adj[adjacency[i][0]][adjacency[i][-1][j].split('.')[0]].append(adjacency[i][-1][j].split('.')[-1])
            else:
                adj[adjacency[i][0]][adjacency[i][-1][j].split('.')[0]].append(adjacency[i][-1][j].split('.')[-1])
#print(adj)

In [5]:
# CONSTRUCTING AND STORING THE ADJACENCY MATRIX.

G = nx.DiGraph()
dfs={}
G.add_nodes_from(states)
for i in range(len(s1)):
    G.add_edge(s1[i][0],s1[i][-1],label=s1[i][1])
if not(os.path.isfile(file9)):
    for i in range(len(states)):
        for j in range(len(states)):
            if states[i]!=states[j]:
                #print(i,j,states[i],states[j])
                dfs[states[i]+'.'+states[j]]=[]
                paths=list(nx.all_simple_paths(G, states[i], states[j]))
                if paths!=[]:
                    for k in range(len(paths)):
                        flag=1
                        for l in range(len(paths[k])-1):
                            if 'tau' in adj[paths[k][l]]:
                                if paths[k][l+1] in adj[paths[k][l]]['tau']:
                                    continue
                                else:
                                    flag=0
                                    break
                            else:
                                flag=0
                                break
                        if flag!=0:
                            dfs[states[i]+'.'+states[j]].append(paths[k])
                            break
else:
    dfs=json.load(open(file9))
# WRITE THE DFS TRAVERSAL TO A FILE FOR FUTURE USE.

if not(os.path.isfile(file9)):
    json.dump(dfs,open(file9,'w'))

In [6]:
# Stutter Adjacency

weak_adj={}
pr_track={}
for i in range(len(states)):
    weak_adj[states[i]]={}
    for j in range(len(actions)):
        if actions[j]!='tau':
            weak_adj[states[i]][actions[j]]=[]
            if actions[j] in adj[states[i]]:
                for k in range(len(adj[states[i]][actions[j]])):
                    if adj[states[i]][actions[j]][k] not in weak_adj[states[i]][actions[j]]:
                        weak_adj[states[i]][actions[j]].append(adj[states[i]][actions[j]][k])
            if 'tau' in adj[states[i]]:
                for state in states:
                    if state!=states[i]:
                        if states[i]+'.'+state in dfs:
                            if actions[j] in adj[state] and dfs[states[i]+'.'+state]!=[]:
                                for l in range(len(dfs[states[i]+'.'+state])):
                                    #print(dfs[states[i]+'.'+state][l])
                                    add=adj[dfs[states[i]+'.'+state][l][-1]][actions[j]]
                                    for m in range(len(add)):
                                        if states[i]+'.'+add[m]+'.'+actions[j] not in pr_track:
                                            pr_track[states[i]+'.'+add[m]+'.'+actions[j]]=dfs[states[i]+'.'+state][l]
                                        if add[m] not in weak_adj[states[i]][actions[j]]:
                                            weak_adj[states[i]][actions[j]].append(add[m])

#print(weak_adj)
#print(pr_track)

In [7]:
# GENERATING THE LTS FILE & INFERRING THE SET OF FINAL STATES. (MCRL2)

final=[]
code1='act '
for i in range(len(actions)):
    code1+=actions[i]
    code1+=','
code1 = code1[:-1]
code1+=';\n\n'
code1+='proc '
for i in range(len(adjacency)):
    if len(adjacency[i][1])>0:
        code1+=adjacency[i][0]+' = '
        for j in range(len(adjacency[i][1])):
            code1+=adjacency[i][1][j]+' + '
        if adjacency[i][0] in NTS:
            for initials in initial:
                code1+=' NEWTRACESTARTING.'+initials+' + '
        code1 = code1[:-2]
        code1+=';\n'
        
    else:
        final.append(adjacency[i][0])
for i in range(len(final)):
    code1+=final[i]+' ='
    for j in range(len(initial)):
        code1+=' NEWTRACESTARTING.'+initial[j]+' +'
    code1=code1[:-1]
    code1+=';\n'   
code1+='init '       
for i in range(len(initial)):
    code1+=initial[i]
    code1+='+'
code1 = code1[:-1]
code1+=';\n\n'
print(code1)
#print(final)

act inviteadditionalreviewer,timeout3,getreview3,decide,getreview2,timeout2,collectreviews,timeout1,getreview1,reject,accept,invitereviewers,timeoutX,getreviewX,NEWTRACESTARTING;

proc s5 = inviteadditionalreviewer.s6 + reject.s4 + accept.s4 ;
s6 = timeoutX.s5 + getreviewX.s5 ;
s10 = timeout3.s8 + getreview3.s8 + getreview2.s0 + timeout2.s0 ;
s8 = getreview2.s3 + timeout2.s3 ;
s12 = decide.s5 ;
s11 = timeout3.s2 + getreview3.s2 + timeout1.s0 + getreview1.s0 ;
s2 = getreview1.s3 + timeout1.s3 ;
s3 = collectreviews.s12 ;
s0 = getreview3.s3 + timeout3.s3 ;
s9 = timeout1.s8 + getreview1.s8 + timeout2.s2 + getreview2.s2 ;
s1 = timeout3.s9 + getreview3.s9 + timeout2.s11 + getreview2.s11 + timeout1.s10 + getreview1.s10 ;
s7 = invitereviewers.s1 ;
s4 = NEWTRACESTARTING.s7 ;
init s7;




In [8]:
if code1!='':
    file = open(file4,'w')
    file.write(code1)
    file.close()

In [9]:
# PREPROCESSING THE TRACE FILE AND INFERRING THE TRACES IN THE LOG.

file = open(file3, 'r')
tracess = file.readlines()
file.close()
trace=[]
ctr=1
traces=[]
traces.append(str(ctr))
event_count=len(tracess)
if event_count>=50000:
    event_count=50000
else:
    event_count=len(tracess)
for i in range(event_count):
    if tracess[i]!='NEWTRACESTARTING\n' and tracess[i]!='tau\n':
        traces.append(tracess[i].rstrip())
    if tracess[i]=='NEWTRACESTARTING\n':
        trace.append(traces)
        ctr+=1
        traces=[]
        traces.append(str(ctr))
print(len(trace))
#print(trace[1])

5525


In [10]:
state_tran_ct={}
for x in adjacency:
    for y in range(len(x[1])):
        state_tran_ct[x[0]+'.'+x[1][y]]=0
state_deg_ct={}
for x in states:
    state_deg_ct[x]=0

In [11]:
# SIMULATING THE TRACES ON THE LST AND CONSTRUCTION OF THE ADTMCS BY POPULATING COUNTS.

good_traces=[]
bad_traces=[]
unsim_traces=[]
for i in range(len(trace)):
    flag=0
    path_old=[]
    sim_flag=1
    for j in range(1,len(trace[i])):
        if len(trace[i])<3 or trace[i][-1] not in end_event:
            flag=1
            break
        if trace[i][j] not in actions:
            flag=1
            break
        if len(path_old)>1000000:
            sim_flag=0
            continue        
        path_new=[]
        if j==1:
            path_old.append(initial[0])
            if weak_adj[initial[0]][trace[i][j]]!=[]:
                for k in range(len(path_old)):
                    for l in range(len(weak_adj[initial[0]][trace[i][j]])):
                        path_new.append(path_old[k]+'.'+weak_adj[initial[0]][trace[i][j]][l])
                path_old=path_new
            else:
                for k in range(len(path_old)):
                    path_new.append(path_old[k]+'.'+'XX')
                    
        else:
            for m in range(len(path_old)):
                a=path_old[m].split('.')[-1]
                if a=='XX':
                    continue
                elif weak_adj[a][trace[i][j]]==[] or len(path_old[m].split('.'))>len(trace[i]):
                    path_new.append(path_old[m]+'.'+'XX')
                else:
                    for n in range(len(weak_adj[a][trace[i][j]])):
                        path_new.append(path_old[m]+'.'+weak_adj[a][trace[i][j]][n])
                path_newp=[]
                list_p=[]
                for pn in range(len(path_new)):
                    ple=path_new[pn].split('.')[-1]
                    if ple not in list_p:
                        list_p.append(ple)
                        path_newp.append(path_new[pn])
                path_new=path_newp
            path_old=path_new
    if sim_flag:
        PATHS=[]
        for path in path_old:
            if path.split('.')[-1]!='XX' and len(path.split('.'))==len(trace[i]):
                PATHS.append(path)
        if PATHS==[]:
            bad_traces.append(trace[i])
        else:
            if len(PATHS)>1:
                print(PATHS,trace[i])
            good_traces.append(trace[i])
            prob_path=PATHS[0]
            prob_path=prob_path.split('.')
            #print(prob_path)
            for o in range(len(prob_path)-1):
                if trace[i][o+1] in adj[prob_path[o]]:
                    if prob_path[o+1] in adj[prob_path[o]][trace[i][o+1]]:
                        state_tran_ct[prob_path[o]+'.'+trace[i][o+1]+'.'+prob_path[o+1]]+=1
                        state_deg_ct[prob_path[o]]+=1
                else:
                    pr_list=pr_track[prob_path[o]+'.'+prob_path[o+1]+'.'+trace[i][o+1]]
                    for p in range(len(pr_list)):
                        if p==len(pr_list)-1:
                            state_tran_ct[pr_list[p]+'.'+trace[i][o+1]+'.'+prob_path[o+1]]+=1
                            state_deg_ct[pr_list[p]]+=1
                        else:
                            state_tran_ct[pr_list[p]+'.'+'tau'+'.'+pr_list[p+1]]+=1
                            state_deg_ct[pr_list[p]]+=1
    elif flag:
        bad_traces.append(trace[i])
    else:
        unsim_traces.append(trace[i])
print(len(good_traces))            
print(len(bad_traces))
print(len(unsim_traces))
#print(state_tran_ct)                  
#print(state_deg_ct)                      

5525
0
0


In [12]:
# INFERRING THE UNIQUE BAD TRACES IN THE FILE. (IF ANY) 

unique_bad_traces=[]
fitness=len(good_traces)/(len(good_traces)+len(bad_traces))*100.0
for x in bad_traces:
    a=''
    for y in range(1,len(x)):
        a+=x[y]+'->'
    a=a[:-2]
    unique_bad_traces.append(a)
ubd={}
for x in unique_bad_traces:
    if x not in ubd:
        ubd[x]=1
    else:
        ubd[x]+=1
        
# GENERATING A TEXT FILE WITH DETAILS OF THE BAD TRACES.

code4=''
if len(ubd)>1:
    code4='\n Total Traces in the file:\t'+str(len(trace))
    code4+='\n Number of Good Traces in the file:\t'+str(len(good_traces))
    code4+='\n Number of Bad Traces in the file:\t'+str(len(bad_traces))
    code4+='\n Number of Unique Bad Trace Types in the file:\t'+str(len(ubd))
    code4+='\n\n Fitness of Model w.r.t. traces it can simulate:\t'+str(round(fitness,4))+'% \n\n'
if len(ubd)>1:
    ctr=1
    code4+='\n Sl No. \t Bad trace \t Frequency\n'
    for k,v in ubd.items():
        code4+='\n'+str(ctr)+'. '+k+'\t'+str(v)
        ctr+=1
    code4+='\n\n Total Number of Unsimulated Traces:\t'+str(len(unsim_traces))
#print(code4)

In [13]:
# CONSTRUCTING THE ADJACENCY MATRIX OF THE ADTMC.

state_tran_ct={}
for x in adjacency:
    for y in range(len(x[1])):
        state_tran_ct[x[0]+'.'+x[1][y]]=0
state_deg_ct={}
for x in states:
    state_deg_ct[x]=0
for i in range(len(good_traces)):
    j=1
    cs=initial[0]
    while j< len(good_traces[i]):
        if good_traces[i][j] in adj[cs] and len(adj[cs][good_traces[i][j]])==1:
            ns=adj[cs][good_traces[i][j]][0]
            state_tran_ct[cs+'.'+good_traces[i][j]+'.'+ns]+=1
            state_deg_ct[cs]+=1
            cs=ns
            j+=1
        elif good_traces[i][j] in adj[cs] and len(adj[cs][good_traces[i][j]])>1:
            print(adj[cs][good_traces[i][j]])
        elif good_traces[i][j] not in adj[cs] and 'tau' in adj[cs] and len(adj[cs]['tau'])==1:
            ns=adj[cs]['tau'][0]
            state_tran_ct[cs+'.'+'tau'+'.'+ns]+=1
            state_deg_ct[cs]+=1
            cs=ns
        elif good_traces[i][j] not in adj[cs] and 'tau' in adj[cs] and len(adj[cs]['tau'])>1:
            leng={}
            for state in states:
                if good_traces[i][j] in adj[state] and dfs[cs+'.'+state]!=[]:
                    leng[state]=len(dfs[cs+'.'+state][0])
            if leng:
                dfs_elem=dfs[cs+'.'+min(leng, key=leng.get)][0]
                for k in range(len(dfs_elem)-1):
                        state_deg_ct[dfs_elem[k]]+=1
                        state_tran_ct[dfs_elem[k]+'.'+'tau'+'.'+dfs_elem[k+1]]+=1
                ns=adj[min(leng, key=leng.get)][good_traces[i][j]][0]
                state_tran_ct[dfs_elem[-1]+'.'+good_traces[i][j]+'.'+ns]+=1
                state_deg_ct[dfs_elem[-1]]+=1
                cs=ns
                j+=1
#print(state_tran_ct)
#print(state_deg_ct)

In [14]:
state_tran_cts = {k:v for k,v in state_tran_ct.items() if v != 0} 
for x in state_tran_cts:
    y=x.split('.')
    state_tran_cts[x]=state_tran_cts[x]/state_deg_ct[y[0]]
#print(state_tran_cts) 
mcrl2_pr_adj={}
for x in state_tran_cts:
    y=x.split('.')
    if y[0] not in mcrl2_pr_adj:
        mcrl2_pr_adj[y[0]]={}
        mcrl2_pr_adj[y[0]][y[1]+'.'+y[2]]=str(Fraction(round(state_tran_cts[x],4)).limit_denominator(100000))
    else:
        mcrl2_pr_adj[y[0]][y[1]+'.'+y[2]]=str(Fraction(round(state_tran_cts[x],4)).limit_denominator(100000))       
#print(mcrl2_pr_adj)    

In [15]:
# GENERATING THE ADTMC FILE. (MCRL2)

proc_comp=[]
ctr=1
code2='act '
for i in range(len(actions)):
    code2+=actions[i]
    code2+=','
code2 = code2[:-1]
code2+=';\n\n'
code2+='proc '
for x in mcrl2_pr_adj:
    proc_comp.append(x)
    y=list(mcrl2_pr_adj[x].keys())
    z=list(mcrl2_pr_adj[x].values())
    #print(z)
    if(len(y)>2):
        code2+=x+' = dist n'+str(ctr)+':Pos['
        for i in range(len(z)):
            code2+='if(n'+str(ctr)+'=='+str(i+1)+','+z[i]+','
        code2+='0'
        for i in range(len(z)):
            code2+=')'
        code2+='].('
        for i in range(len(y)):
            code2+='(n'+str(ctr)+'=='+str(i+1)+')->'+y[i]+'+'
        code2 = code2[:-1]
        code2+=');\n'
        ctr+=1
    elif(len(y)==2):
        code2+=x+' = dist b'+str(ctr)+':Bool[if(b'+str(ctr)+','+z[0]+','+z[1]+')'
        code2+='].('
        code2+='b'+str(ctr)+'->'+y[0]+' + '
        code2+='!b'+str(ctr)+'->'+y[1]
        code2+=');\n'
        ctr+=1
    else:
        code2+=x+' = dist b'+str(ctr)+':Bool[if(b'+str(ctr)+',1,0)].(b'+str(ctr)+'->'+y[0]+');\n'
        ctr+=1 
a=list(set(states)-set(proc_comp))
for x in a:
    code2+=x+' = dist b'+str(ctr)+':Bool[if(b'+str(ctr)+',1,0)].(b'+str(ctr)+'->tau.'+x+');\n'
    ctr+=1
code2+='init '       
for i in range(len(initial)):
    code2+=initial[i]
    code2+='+'
code2 = code2[:-1]
code2+=';\n\n'
#print(code2)

In [16]:
# CONVERTING THE ADTMC TO SDTMC USING THE MODEL EMBEDDING ats FOR PROBABILISTIC SYSTEMS.

prism_pr_adj={}
for x in state_tran_cts:
    y=x.split('.')
    if y[0] not in prism_pr_adj:
        prism_pr_adj[y[0]]={}
        prism_pr_adj[y[0]][y[1]+'.'+y[2]]=round(state_tran_cts[x],6)
    else:
        prism_pr_adj[y[0]][y[1]+'.'+y[2]]=round(state_tran_cts[x],6)      
#print(prism_pr_adj)
#print(len(prism_pr_adj))
labels={}
mapping={}
ctr=1
for x in prism_pr_adj:
    a=x
    b=prism_pr_adj[x]
    #print(a,b)
    if a not in labels:
        labels[a]='bot'
    if a not in mapping:
        mapping[a]=a
    for y in b:
        if y not in mapping and y.split('.')[0]!='tau':
            mapping[y]='s'+str(len(states)+ctr)
            ctr+=1
        if y not in labels and y.split('.')[0]!='tau':
            labels[mapping[y]]=y.split('.')[0]
#print(labels)
ats_adj={}
for x in prism_pr_adj:
    if x not in ats_adj:
        ats_adj[x]={}
        for y in prism_pr_adj[x]:
            if y.split('.')[0]!='tau':
                ats_adj[x][mapping[y]]=prism_pr_adj[x][y]
            else:
                ats_adj[x][y.split('.')[-1]]=prism_pr_adj[x][y]
for x,y in mapping.items():
    if x!=y:
        ats_adj[y]={x.split('.')[-1]:1.0}
#print(ats_adj,initial)
deadlock=list(set(states)-(set(ats_adj.keys())&set(states)))
for x in deadlock:
    labels[x]='bot'
    mapping[x]=x
#print(deadlock)

In [17]:
# GENERATING THE SDTMC CODE. (PRISM)

code3=''
code3+='\n\n dtmc \n\t module ats_ADTMC \n\n //Variables\n'
code3+='s:[0..'+str(len(mapping))+'] init '+initial[0].split('s')[-1]
code3+=';\n'
for a,b in ats_adj.items():
    code3+='\n [] s='+a.split('s')[-1]+' ->'
    for c in b:
        code3+=' '+str(b[c])+":(s'="+c.split('s')[-1]+')+'
    code3=code3[:-1]
    code3+=';'
for x in deadlock:
    code3+='\n [] s='+x.split('s')[-1]+" -> 1.0:(s'="+x.split('s')[-1]+');'
code3+='\n\n\t endmodule \n\n //Labels'
code3+='\n label "Initial" = (s='+initial[0].split('s')[-1]+');'
code3+='\n label "bot"= '
for i in labels:
    if labels[i]=='bot':
        code3+='(s='+i.split('s')[-1]+')|'
code3=code3[:-1]
code3+=';'
for i in range(len(actions)):
    flag=0
    for j in labels:
        if labels[j]==actions[i]:
            flag=1
            break
    if flag==1:
        code3+='\n label "'+actions[i]+'"= '
        for j in labels:
            if labels[j]==actions[i]:
                code3+='(s='+j.split('s')[-1]+')|'
        code3=code3[:-1]
        code3+=';'
    else:
        continue
code3+='\n\n //Rewards' 
code3+='\n rewards "original_step"\n'
for i in labels:
    if labels[i]=='bot':
        code3+='(s='+i.split('s')[-1]+')'+'|'
code3=code3[:-1]
code3+=': 1;\n endrewards\n'
for i in range(len(actions)):
    flag2=0
    for j in labels:
        if labels[j]==actions[i]:
            flag2=1
            break
    if flag2==1:
        code3+='\n rewards "'+actions[i]+'"\n'
        for j in labels:
            if labels[j]==actions[i]:
                code3+='(s='+j.split('s')[-1]+')'+'|'
        code3=code3[:-1]
        code3+=': 1;\n endrewards\n'
    else:
        continue
code3+='\n\n //State Mapping'
for x,y in mapping.items():
    code3+='\n // '+x+' -> '+y
#print(code3)

In [18]:
# WRITING ALL THE INFORMATION AND CODES TO FILES.

if code2!='':
    file = open(file6,'w')
    file.write(code2)
    file.close()
if code3!='':
    file = open(file7,'w')
    file.write(code3)
    file.close()